In [28]:
import pandas as pd
from pathlib import Path

In [29]:
# Define root and folders
project_root = Path().resolve().parents[1]  # to root directory
raw_dir = project_root / 'data' / 'raw'
processed_dir_to_save = project_root / 'data' / 'processed'
processed_dir_to_save.mkdir(parents=True, exist_ok=True)

SAVE_PATH = processed_dir_to_save / 'merged_prices.csv'

In [30]:

# Load raw CSVs
btc = pd.read_csv(raw_dir / 'bitcoin_historical_dataset_raw.csv')
gold = pd.read_csv(raw_dir / 'gold_prices_raw.csv', parse_dates=['Date'])
sp500 = pd.read_csv(raw_dir / 'sp500_prices_raw.csv', parse_dates=['Date'])
oil = pd.read_csv(raw_dir / 'oil_prices_raw.csv', parse_dates=['Date'])

In [31]:
btc.head()

,Start,End,Open,High,Low,Close,Volume,Market Cap
0,01/06/2025,NaN,NaN,NaN,NaN,104709.00,NaN,NaN
1,01/05/2025,NaN,NaN,NaN,NaN,94237.00,NaN,NaN
2,01/04/2025,NaN,NaN,NaN,NaN,82461.00,NaN,NaN
3,01/03/2025,08/03/2025,84307.46,94808.02,81942.81,86832.99,8.757380e+10,1.740000e+12
4,01/02/2025,01/03/2025,102318.00,102760.00,78534.47,84501.01,6.309437e+10,1.900000e+12


In [32]:
# First, parse the dates correctly specifying the current format
btc['Start'] = pd.to_datetime(btc['Start'], format='%d/%m/%Y')
# Then convert to string in the new format YYYY-mm-dd if needed
btc['Start'] = btc['Start'].dt.strftime('%Y-%m-%d')

In [33]:
btc.rename({'Start':'Date', 'Close':'BTC_PRICE_$'}, axis=1, inplace=True)

In [34]:
btc = btc[['Date', 'BTC_PRICE_$']]

In [35]:
btc.tail()

,Date,BTC_PRICE_$
181,2010-05-01,0.030
182,2010-04-01,0.020
183,2010-03-01,0.010
184,2010-02-01,0.005
185,2010-01-01,0.000


In [36]:
gold.head(10)

,Date,Gold_Price_$
0,2010-01-01,1123.6
1,2010-02-01,1159.6
2,2010-03-01,1154.3
3,2010-04-01,1223.2
4,2010-05-01,1260.3
5,2010-06-01,1290.0
6,2010-07-01,1224.3
7,2010-08-01,1294.3
8,2010-09-01,1355.7
9,2010-10-01,1405.6


In [37]:
oil.head(20)

,Date,Oil_Price_USD
0,2010-01-01,72.9
1,2010-02-01,79.7
2,2010-03-01,83.8
3,2010-04-01,86.2
4,2010-05-01,74.0
5,2010-06-01,75.6
6,2010-07-01,78.9
7,2010-09-01,80.0
8,2010-10-01,81.4
9,2010-11-01,84.1


In [38]:
oil.rename({'Oil_Price_USD': 'Oil_Price_$'}, axis=1, inplace=True)

In [39]:
btc['Date'] = pd.to_datetime(btc['Date'])
gold['Date'] = pd.to_datetime(gold['Date'])
sp500['Date'] = pd.to_datetime(sp500['Date'])
oil['Date'] = pd.to_datetime(oil['Date'])

In [40]:
# Merge on 'date'
merged = pd.merge(btc, gold, on='Date', how='outer')
merged = pd.merge(merged, sp500, on='Date', how='outer')
merged = pd.merge(merged, oil, on='Date', how='outer')

In [41]:
merged

,Date,BTC_PRICE_$,Gold_Price_$,S&P500_$,Oil_Price_$
0,2010-01-01,0.000,1123.6,1073.9,72.9
1,2010-02-01,0.005,1159.6,1104.5,79.7
2,2010-03-01,0.010,1154.3,1169.4,83.8
3,2010-04-01,0.020,1223.2,1186.7,86.2
4,2010-05-01,0.030,1260.3,1089.4,74.0
...,...,...,...,...,...
182,2025-03-01,86832.990,3053.9,5611.9,71.5
183,2025-04-01,82461.000,3220.3,5569.1,58.2
184,2025-05-01,94237.000,3218.2,5911.7,60.8
185,2025-06-01,104709.000,3305.1,6033.1,NaN


In [42]:
# Sort by date
merged_df = merged.sort_values('Date').reset_index(drop=True)

In [43]:
merged_df.head(30)

,Date,BTC_PRICE_$,Gold_Price_$,S&P500_$,Oil_Price_$
0,2010-01-01,0.0000,1123.6,1073.9,72.9
1,2010-02-01,0.0050,1159.6,1104.5,79.7
2,2010-03-01,0.0100,1154.3,1169.4,83.8
3,2010-04-01,0.0200,1223.2,1186.7,86.2
4,2010-05-01,0.0300,1260.3,1089.4,74.0
5,2010-06-01,0.0400,1290.0,1030.7,75.6
6,2010-07-01,0.0679,1224.3,1101.6,78.9
7,2010-08-01,0.0600,1294.3,1049.3,NaN
8,2010-09-01,0.0619,1355.7,1141.2,80.0
9,2010-10-01,0.1925,1405.6,1183.3,81.4


In [44]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          187 non-null    datetime64[ns]
 1   BTC_PRICE_$   186 non-null    float64       
 2   Gold_Price_$  186 non-null    float64       
 3   S&P500_$      186 non-null    float64       
 4   Oil_Price_$   160 non-null    float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 7.4 KB


In [45]:
# some oil values are missing (prob due to 1st being a weekend). Fill with average of previous and next.
# Create a rolling average (centered on the row)
def fill_nan_with_neighbor_avg(series):
    new_series = series.copy()
    for i in range(1, len(series) - 1):
        if pd.isna(series[i]):
            above = series[i - 1]
            below = series[i + 1]
            if pd.notna(above) and pd.notna(below):
                new_series[i] = (above + below) / 2
    return new_series

# Apply this only to Oil_Price_$
merged_df['Oil_Price_$'] = fill_nan_with_neighbor_avg(merged_df['Oil_Price_$'])

In [46]:
merged_df.head(20)

,Date,BTC_PRICE_$,Gold_Price_$,S&P500_$,Oil_Price_$
0,2010-01-01,0.0000,1123.6,1073.9,72.90
1,2010-02-01,0.0050,1159.6,1104.5,79.70
2,2010-03-01,0.0100,1154.3,1169.4,83.80
3,2010-04-01,0.0200,1223.2,1186.7,86.20
4,2010-05-01,0.0300,1260.3,1089.4,74.00
5,2010-06-01,0.0400,1290.0,1030.7,75.60
6,2010-07-01,0.0679,1224.3,1101.6,78.90
7,2010-08-01,0.0600,1294.3,1049.3,79.45
8,2010-09-01,0.0619,1355.7,1141.2,80.00
9,2010-10-01,0.1925,1405.6,1183.3,81.40


In [48]:
# Save 
merged_df.to_csv(SAVE_PATH, index=False)